In [1]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [2]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [3]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 30
n_iter = 13000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [4]:
import impl.RNN as nn
net = nn.RNN(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.LSTM as nn
# net = nn.LSTM(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU as nn
# net = nn.GRU(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU2 as nn
# net = nn.GRU2(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# import impl.GRU3 as nn
# net = nn.GRU3(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

# # GRU4 goes to inf because the res-conn is gone.
# import impl.GRU4 as nn
# net = nn.GRU4(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [5]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step, # This is the width of the network
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.3513
he us lJofcn manwwone ial pf che dofa.tlRx.Evn Ahe wNin an lin loudsorin chuind Ihendadea, Ohpec iou


Iter-2000 loss: 2.5747
tt.e cat iap raodoi. thisse m8re oxpani as iotGla wor da%a.a3Japdrcmestis bhn e the Jary 's torof th


Iter-3000 loss: 2.0704
war y'sticiud re the murt. Thempese sint y's warat , wh ow Cf rond wedercgince"otrop oilal 10envored


Iter-4000 loss: 1.7557
s. The meland Ahesta Obolt. on thk dy. The fourtn. ;he tore thile to" destrea, thinal conte yorer an


Iter-5000 loss: 1.5442
 Jd an s Ruroly, and Ranghind so ordpben , verot or oh Japan's torta inal 's tareg by fd farke 1ry 8


Iter-6000 loss: 1.3885
0th enstired if osa cithe an ares t of sne fer hest. unes as pen apeachas earixshimrlecturod ooke Er


Iter-7000 loss: 1.2549
ded acesoris duron and anderind ineder estity unt sillHigstaiac-kagutu, wati A ioly Chist erdeg the 


Iter-8000 loss: 1.1500
 he Nipetin ty teo Japanu,sh ch thi", andetorin Es.and Japan"ss marledtrrys peomithe "s. The we

In [7]:
=========================================================================
Iter-299000 loss: 0.0278
=========================================================================
s. Japan is a developed country with a high standard of living and Human Development Index whose pop
=========================================================================


=========================================================================
Iter-300000 loss: 0.0268
=========================================================================
apan to open to the West. Nearly two decades of internal conthic-komy parionory. From the 12th centu
=========================================================================


Out[5]:
<impl.GRU.GRU at 0x7f1401bc5588>

SyntaxError: invalid syntax (<ipython-input-7-b8c9dab95f4c>, line 1)

In [8]:
=========================================================================
Iter-299000 loss: 0.0914
=========================================================================
s. Japan's history. From the 12th century until 1868, Japan's n7meamean Laas pan hs, Japan was men M
=========================================================================


=========================================================================
Iter-300000 loss: 0.1100
=========================================================================
apan was inhabited as early as the Upper Paleolithic period. The first are Honsh mict of inter of in
=========================================================================


Out[10]:
<impl.LSTM.LSTM at 0x7f51924f1940>

SyntaxError: invalid syntax (<ipython-input-8-b3108e7b52a0>, line 1)

In [9]:
=========================================================================
Iter-299000 loss: 0.1178
=========================================================================
s. Japan is a developed country wino avinsm itso. The foulslint of is in the namiort tites Toky byde
=========================================================================


=========================================================================
Iter-300000 loss: 0.0913
=========================================================================
apan was ruled in the name of the Emperor as a d bevere Hf Hiros. Japan is athe world's third-s. Jap
=========================================================================


Out[8]:
<impl.RNN.RNN at 0x7f51924f1630>

SyntaxError: invalid syntax (<ipython-input-9-b244120dc262>, line 1)